# Dataset Usage Example (using Marketsplit)

This example demonstrates how to work with datasets in OMMX Quantum Benchmarks using Marketsplit as a representative case.

**Important**: The patterns shown here apply to ALL datasets in the collection (Labs, Portfolio, Topology, etc.). Simply replace `Marketsplit` with any other dataset class name - the API is identical across all datasets.

## Dataset Overview

In [2]:
from ommx_quantum_benchmarks.qoblib import Marketsplit

# Initialize the dataset
dataset = Marketsplit()

print(f"Dataset: {dataset.name}")
print(f"Description: {dataset.description}")
print(f"Available models: {dataset.model_names}")

# Check available instances
for model, instances in dataset.available_instances.items():
    print(f"{model}: {len(instances)} instances")

Dataset: 01_marketsplit
Description: Marketsplit dataset in ommx format, originally provided by https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/tree/main/01-marketsplit?ref_type=heads.
Available models: ['binary_linear', 'binary_unconstrained']
binary_linear: 156 instances
binary_unconstrained: 156 instances


## Working with Binary Linear Model

In [ ]:
# Load a specific instance
model_name = "binary_linear"
instance_name = "ms_03_050_002"

instance, solution = dataset(model_name, instance_name)

print(f"Loaded instance: {instance_name}")
print(f"Instance type: {type(instance)}")
print(f"Solution available: {solution is not None}")

if solution:
    print(f"Objective value: {solution.objective}")
    print(f"Feasible: {solution.feasible}")
    print(f"Number of variables: {len(solution.state.entries)}")

## Solution Verification

In [ ]:
if solution is not None:
    # Evaluate the solution using the instance
    evaluated = instance.evaluate(solution.state)
    
    print("Solution Verification:")
    print(f"Original objective: {solution.objective}")
    print(f"Evaluated objective: {evaluated.objective}")
    print(f"Objectives match: {solution.objective == evaluated.objective}")
    
    print(f"Original feasibility: {solution.feasible}")  
    print(f"Evaluated feasibility: {evaluated.feasible}")
    print(f"Feasibility matches: {solution.feasible == evaluated.feasible}")
    
    # Check state consistency
    state_match = solution.state.entries == evaluated.state.entries
    print(f"States match: {state_match}")